In [1]:
#pip install -q -U transformers datasets accelerate peft trl bitsandbytes wandb sentence_transformers faiss-gpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [2]:
import warnings
warnings.simplefilter('ignore')
import os
import argparse
import accelerate
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model


2024-03-26 16:50:33.809409: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 16:50:33.809511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 16:50:33.941916: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from kaggle_secrets import UserSecretsClient
import wandb
wandb_api = "your_wandb_api_key"
wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
#dataset = 'ardaorcun/turkish-instruction-dataset-prepared'
dataset = 'merve/turkish_instructions'
base_model = 'ytu-ce-cosmos/turkish-gpt2-large'
model_name = 'cosmos-finetuned'
auth_token = 'your_huggingface_token'
#push_to_hub = False

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

        

def format_instruction(sample):
    return f"""Sen cevap vermeyi seven yardımcı bir dil modelisin.
        ### Input:
        {sample["talimat"]}
        
        ### Context:
        {sample[" giriş"]}

        ### Response:
        {sample[" çıktı"]}
    """

In [6]:
dataset = load_dataset(dataset, token=auth_token)
# base model to finetune
model_id = base_model

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset['train'] = dataset['train'].remove_columns(['Unnamed: 0'])

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['talimat', ' giriş', ' çıktı'],
        num_rows: 51563
    })
})

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             use_cache=False,
                                             device_map="auto")

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [11]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=1280, out_features=3840, bias=True)
          (c_proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=1280, out_features=5120, bias=True)
          (c_proj): Linear4bit(in_features=5120, out_features=1280, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, element

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [13]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['c_proj',
                    'c_fc',
                    'gate_proj',
                    'c_proj',
                    'c_attn'],
    bias="lora_only",
    use_rslora=True,
    fan_in_fan_out=True,
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)


# print the number of trainable model params
print_trainable_parameters(model)

trainable params: 12211200 || all params: 431932160 || trainable%: 2.827110627743023


In [14]:
new_model = 'final_moodel'

In [15]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    eval_steps=1500,
    logging_steps=1,
    optim="paged_adamw_8bit",
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    warmup_steps=10,
    report_to="wandb",
    #max_steps=25, # Remove this line for a real fine-tuning
)
model_args = TrainingArguments(
    output_dir="cosmos-deneme-22/04(0)",
    num_train_epochs=5,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="paged_lion_8bit",
    logging_steps=11,
    save_strategy="epoch",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="linear",
    disable_tqdm=False,
    #max_steps=10 # disable line for full time training 
)

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    #peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=model_args,
)



# train
trainer.train()
trainer.model.save_pretrained(new_model)

Generating train split: 0 examples [00:00, ? examples/s]

wandb: Currently logged in as: arda-orcun (data-leak). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240326_165157-mq87k92o
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run classic-wind-32
wandb: ⭐️ View project at https://wandb.ai/data-leak/huggingface
wandb: 🚀 View run at https://wandb.ai/data-leak/huggingface/runs/mq87k92o/workspace


Step,Training Loss
11,60.268300
22,62.161200
33,56.601300
44,6.667000
55,1.991300
66,1.907200
77,1.851600
88,1.822400
99,1.816800
110,1.804300


In [17]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1280)
        (wpe): Embedding(1024, 1280)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-35): 36 x GPT2Block(
            (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1280, out_features=3840, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1280, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3840, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (

In [22]:
del model 
#del pipe 
del trainer 
import gc 
gc.collect()
gc.collect()

0

In [25]:
from peft import PeftModel

In [26]:
model = AutoModelForCausalLM.from_pretrained(base_model,
                            low_cpu_mem_usage=True,
                            return_dict=True,
                            torch_dtype=torch.float16,
                            device_map="auto")

model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="rigt"

In [31]:
model_name='finetuned_cosmos2603'
model.push_to_hub(model_name, token = auth_token)
tokenizer.push_to_hub(model_name, token=auth_token)

model.safetensors:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ardaorcun/finetuned_cosmos2603/commit/ec52ab81283c750d6270414d6c88c936f1363ae7', commit_message='Upload tokenizer', commit_description='', oid='ec52ab81283c750d6270414d6c88c936f1363ae7', pr_url=None, pr_revision=None, pr_num=None)